## Converting STREUSLE Confidence Scores

1. Load all.csv for STREUSLE dev.
2. Change all predicted and ground truth labels to UPOS.
3. Sum relevant confidence scores for new tagset.

In [1]:
# Notebook starts in notebooks folder. Change working directory back to streusle-tagger
%cd ../../../

C:\michael\dev\streusle-tagger


In [2]:
import pickle

import pandas as pd

In [3]:
dataset = "test"
streusle_original = pd.read_csv(f"calibration/confidence_scores/{dataset}/all.csv")
streusle_original.head()

,Unnamed: 0,Tokens,Ground Truth,Ground Truth Indexes,Predicted Tags,Predicted Tag Indexes,0,1,2,3,...,586,587,588,589,590,591,592,593,594,595
0,0,Highly,O-ADV,5,B-ADV,50,6.117443e-07,2.637536e-07,2.411689e-07,0.001880,...,1.667421e-11,1.115616e-12,3.396094e-11,3.191509e-11,1.331150e-11,2.616954e-12,3.688850e-12,3.859260e-11,1.164130e-11,1.286602e-11
1,1,recommended,O-V-v.communication,13,I~-V-v.communication,82,2.251189e-08,4.008616e-09,8.058923e-05,0.000012,...,1.984751e-11,2.298898e-11,1.430627e-11,9.483878e-12,1.960824e-11,3.470025e-11,2.913311e-11,1.411671e-11,2.569076e-11,4.352270e-11
2,0,My,O-PRON.POSS-p.SocialRel|p.Gestalt,34,O-PRON.POSS-p.SocialRel|p.Gestalt,34,7.755846e-07,6.987049e-06,2.462810e-06,0.000075,...,7.356474e-11,8.683496e-12,2.910608e-10,1.151461e-10,1.139141e-10,2.518833e-11,1.346405e-11,1.549520e-10,8.338267e-11,6.697124e-11
3,1,8,O-NUM,18,O-NUM,18,1.639997e-06,2.275625e-06,3.184332e-05,0.000822,...,6.832474e-10,4.360681e-10,1.105464e-09,3.053248e-10,1.583515e-09,7.633524e-10,6.114179e-10,4.821608e-10,8.428808e-10,6.535229e-10
4,2,year,B-ADJ,36,B-N-n.PERSON,43,1.839418e-04,1.061353e-06,3.827731e-03,0.000238,...,3.078570e-09,4.407594e-09,2.981440e-09,1.320173e-09,4.721484e-09,4.167204e-09,5.503994e-09,5.213304e-09,2.294336e-09,8.021656e-09


In [4]:
streusle_consolidated_path = f"calibration/confidence_scores/{dataset}/all_consolidated.csv"

with open("calibration/consolidated_labels.pickle", "rb") as f:
    # Map of each new consolidated label to the corresponding indexes of original labels
    new_labels = pickle.load(f)

In [5]:
streusle_consolidated_path = f"calibration/confidence_scores/{dataset}/all_consolidated_no_lexcat.csv"

with open("calibration/consolidated_labels_no_lexcat.pickle", "rb") as f:
    # Map of each new consolidated label to the corresponding indexes of original labels
    new_labels = pickle.load(f)

In [6]:
streusle_consolidated_columns = ["Token Index", "Tokens", "Predicted Tag", "Predicted Index", "Ground", "Ground Index"] + list(new_labels.keys())
streusle_consolidated = pd.DataFrame(columns=streusle_consolidated_columns)

streusle_consolidated["Token Index"] = streusle_original["Unnamed: 0"]
streusle_consolidated["Tokens"] = streusle_original["Tokens"]

In [7]:
old_index_to_new_label = {}
for k, v in new_labels.items():
    for num in v:
        old_index_to_new_label[num] = k

In [8]:
new_labels_list = list(new_labels)
new_label_to_new_index = dict(zip(new_labels_list, list(range(len(new_labels_list)))))

# Get predicted labels and indexes using consolidated labelset
predicted_labels_consolidated = []
predicted_indexes_consolidated = []

for index in streusle_original["Predicted Tag Indexes"]:
    new_label = old_index_to_new_label[index]
    predicted_labels_consolidated.append(new_label)
    new_index = new_label_to_new_index[new_label]
    predicted_indexes_consolidated.append(new_index)

In [9]:
# Get ground labels and indexes using consolidated labelset
ground_labels_consolidated = []
ground_indexes_consolidated = []

for index in streusle_original["Ground Truth Indexes"]:
    new_label = old_index_to_new_label[index]
    ground_labels_consolidated.append(new_label)
    new_index = new_label_to_new_index[new_label]
    ground_indexes_consolidated.append(new_index)

In [10]:
streusle_consolidated["Predicted Tag"] = predicted_labels_consolidated
streusle_consolidated["Predicted Index"] = predicted_indexes_consolidated
streusle_consolidated["Ground"] = ground_labels_consolidated
streusle_consolidated["Ground Index"] = ground_indexes_consolidated

In [11]:
def sum_scores(new_label, row_index):
    score_sum = 0
    
    for i in new_labels[new_label]:
        score_sum += streusle_original[str(i)][row_index]
        
    return score_sum

In [12]:
score_column_names = list(new_labels.keys())

In [13]:
score_columns = {}
for c in score_column_names:
    score_columns[c] = []
for i, row in streusle_consolidated.iterrows():
    if i % 1000 == 0:
        print(i)
    for c in score_column_names:
        score_columns[c].append(sum_scores(c, i))

0
1000
2000
3000
4000
5000


In [14]:
for c in score_column_names:
    streusle_consolidated[c] = score_columns[c]

In [15]:
streusle_consolidated.head()

,Token Index,Tokens,Predicted Tag,Predicted Index,Ground,Ground Index,O,I,O-v.stative,O-n.group,...,B-??,B-n.plant,o-v.communication,o-v.emotion,b-n.substance,B-n.other,b-v.possession,b-n.quantity,o-n.naturalobject,b-n.location
0,0,Highly,B,23,O,0,0.002034,0.000130,1.556116e-06,6.488303e-07,...,2.921197e-10,5.666673e-11,1.620164e-10,4.195320e-11,1.138003e-10,2.419300e-11,9.309886e-10,1.990661e-12,1.299592e-11,3.688850e-12
1,1,recommended,I,1,O-v.communication,7,0.000013,0.999509,1.966777e-06,2.292698e-07,...,2.772937e-11,1.126808e-10,1.861066e-11,1.801614e-10,2.461519e-10,9.916642e-11,3.654432e-10,7.515632e-11,4.174218e-10,2.913311e-11
2,0,My,O,0,O,0,0.999944,0.000003,7.453182e-06,2.694014e-06,...,4.181791e-09,1.383223e-09,8.502502e-10,1.166226e-10,1.736969e-10,1.875254e-11,6.813698e-10,2.316337e-11,9.126617e-11,1.346405e-11
3,1,8,O,0,O,0,0.982591,0.000043,7.247547e-08,1.487141e-06,...,1.337933e-08,1.195734e-08,1.092339e-08,1.373391e-09,6.967900e-09,4.782006e-10,4.817236e-09,2.856571e-07,1.422800e-08,6.114179e-10
4,2,year,B-n.person,25,B,23,0.000552,0.004217,3.680640e-05,1.448923e-05,...,1.256402e-08,1.012066e-07,2.362576e-08,7.782393e-09,2.644042e-08,7.196902e-10,7.066464e-08,4.091425e-08,1.279801e-07,5.503994e-09


In [16]:
streusle_consolidated.to_csv(streusle_consolidated_path, index=False)